## GENETIC PROGRMAMMING 

### DO NOT USE FIBONACCI OR MICHAEL WILL CRY ;_;

In [1]:
# A002064 		Cullen numbers: a(n) = n*2^n + 1. 
# Should I generate a dataset like in the example code or should I provide a dataset obtained from OEIS??

In [2]:
# CODE ADAPTED FROM TUTORIAL.

In [3]:
from random import random, randint, seed
from statistics import mean
from copy import deepcopy
import numpy as np

In [4]:
# Default Parameters - these are changed lcater...
POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 100  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 
# note: might change so that large tree size is discouraged rather than rejected, i.e. has an -ve effect on fitness that isn't setting = 0
MAX_TREE_SIZE   = 10   # max size of tree

In [5]:
# Default Non terminals:
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
FUNCTIONS = [add, sub, mul]

In [6]:
# Default Terminals:
TERMINALS = ['x', -2, -1, 0, 1, 2]
#TERMINALS = ['x', 0, 1, 2]

In [7]:
# Cullen numbers dataset:
def cullen_number(n):
    return n*(2**n) + 1

def cube_seq(n):
    return 32*n - 2*n**2 + 7*n**3

def generate_dataset(): # generate 101 data points from target_func
    dataset = []
    # changed this from original to be only natural numbers (>=0)
    for x in range(0,101):
        x /= 100
        dataset.append([x, cullen_number(x)])
    print("dataset generated")
    return dataset


In [8]:
# Generic progrma class:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        self.fitness = 0
        # generation_found is only set for the best tree found over a run for use when running experiments
        self.generation_found = 0
        
    def set_fitness(self, new_fitness):
        self.fitness = new_fitness
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def compute_tree(self, x): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif self.data == 'x': return x
        else: return self.data
            
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)

    def mutation(self):
        if random() < PROB_MUTATION: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation()
        elif self.right: self.right.mutation() 

    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r

    def build_subtree(self): # count is list in order to pass "by reference"
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.build_subtree()
        if self.right: t.right = self.right.build_subtree()
        return t
                        
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.build_subtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret

    def crossover(self, other): # xo 2 trees at random nodes
        if random() < XO_RATE:
            second = other.scan_tree([randint(1, other.size())], None) # 2nd random subtree
            self.scan_tree([randint(1, self.size())], second) # 2nd subtree "glued" inside 1st tree

In [9]:
# FITNESS AND SELECTION

# MODIFIED FROM OG!
def fitness(individual, dataset): # inverse mean absolute error over dataset normalized to [0,1]
    #if power function would cause an overflow, we declare the tree unfit, with fitness = 0
    # set permanent max solution tree size here... i.e. fitness = 0 when depth > max_depth (25)
    if individual.size() > MAX_TREE_SIZE:
        result = 0
    else:
        try:
            result = 1 / (1 + mean([abs(individual.compute_tree(ds[0]) - ds[1]) for ds in dataset]))
        except OverflowError:
            result = 0
    return result
    #return 1 / (1 + mean([abs(individual.compute_tree(ds[0]) - ds[1]) for ds in dataset]))

# USE THIS FITNESS FUNCTION TO TURN THE PROGRAM INTO ESSENTIALLY A RANDOM SEARCH
# def fitness(individual, dataset):
#     try:
#         result = 1 / (1 + mean([abs(individual.compute_tree(ds[0]) - ds[1]) for ds in dataset]))
#     except OverflowError:
#         result = 0
#     if result == 1:
#         return 1
#     else:
#         return 0

# tournament selection
# possible todo: change to roulette wheel selection (or other kind of selection and observe results)
def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = [fitnesses[tournament[i]] for i in range(TOURNAMENT_SIZE)]
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 


In [10]:
# INITIALISE POPULATION
def init_population(): # ramped half-and-half
    pop = []
    for md in range(3, MAX_DEPTH + 1):
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = True, max_depth = md) # grow
            pop.append(t) 
        for i in range(int(POP_SIZE/6)):
            t = GPTree()
            t.random_tree(grow = False, max_depth = md) # full
            pop.append(t) 
    return pop

In [11]:
# MAIN LOOP:
def main():
    dataset = generate_dataset()
    population = init_population() 
    best_of_run = None
    best_of_run_f = 0
    best_of_run_gen = 0
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    # go evolution!
    for gen in range(GENERATIONS):        
        nextgen_population=[]
        for i in range(POP_SIZE):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            parent1.crossover(parent2)
            parent1.mutation()
            nextgen_population.append(parent1)
        population=nextgen_population
        fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
        #printing generational results and new trees every time new best is found:
        print("gen:", gen)
        if max(fitnesses) > best_of_run_f:
            best_of_run_f = max(fitnesses)
            best_of_run_gen = gen
            best_of_run = deepcopy(population[fitnesses.index(max(fitnesses))])
            best_of_run.set_fitness(best_of_run_f)
            print("________________________")
            print("gen:", gen, ", best_of_run_f:", round(max(fitnesses),3), ", best_of_run:") 
            best_of_run.print_tree()
        if best_of_run_f == 1: break   

    print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
            " and has f=" + str(round(best_of_run_f,3)))
    best_of_run.print_tree()
    # returning best_of_run and gen here purely for tuning / graphing purposes
    #return best_of_run
    return gen

In [12]:
# POP_SIZE        = 60   # population size
# MIN_DEPTH       = 2    # minimal initial random tree depth
# MAX_DEPTH       = 5    # maximal initial random tree depth
# GENERATIONS     = 100  # maximal number of generations to run evolution
# TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
# XO_RATE         = 0.8  # crossover rate 
# PROB_MUTATION   = 0.2  # per-node mutation probability 
# TERMINALS       = ['x', -2, -1, 0, 1, 2]
GENERATIONS = 250
main()

GENERATIONS = 100

dataset generated
gen: 0
________________________
gen: 0 , best_of_run_f: 0.001 , best_of_run:
add
   add
      -2
      add
         2
         x
   add
      2
      x
gen: 1
gen: 2
________________________
gen: 2 , best_of_run_f: 0.001 , best_of_run:
sub
   mul
      x
      x
   mul
      0
      -2
gen: 3
gen: 4
________________________
gen: 4 , best_of_run_f: 0.001 , best_of_run:
add
   x
   add
      mul
         x
         x
      add
         x
         -2
gen: 5
________________________
gen: 5 , best_of_run_f: 0.001 , best_of_run:
mul
   x
   add
      x
      2
gen: 6
gen: 7
________________________
gen: 7 , best_of_run_f: 0.001 , best_of_run:
sub
   mul
      x
      mul
         x
         x
   sub
      -2
      x
gen: 8
gen: 9
________________________
gen: 9 , best_of_run_f: 0.001 , best_of_run:
sub
   mul
      x
      mul
         x
         mul
            x
            x
   x
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen:

In [13]:
# TUNED GP...

# this can get a fitness of 1!

# GP DESIGN CHOICES:
# ADDED POWER FUNCTION, CHANGED FITNESS FUNCTION TO REJECT POWERS THAT ARE TOO LARGE (and cause an overflow error), AND TO REJECT TREES LARGER THAN 25 NODES

# PARAM CHOICES:
# Large number of generations as it's been shown to take some time to converge!
GENERATIONS = 250
# to increase exploration... in report should mention that I was struggling with non convergent solutions, that were getting stuck in local minima
# so I wanted to increase exploration a bit to help me avoid this
# This does mean the algorithm is closer to random search though... so maybe loses some of the benefits of GP
PROB_MUTATION = 0.4 # also works well with 0.2 and 0.3
XO_RATE = 0.8 #might change to 0.6
# to reduce search space, and execution time
MAX_TREE_SIZE = 25
# to increase parallelism, and give the alg more chance of starting in favourable positions so I don't need to run multiple times to get a f=1
POP_SIZE = 120

# a valid tree for 'x * (2**x) + 1' is: (size=7)
# add
#   mul
#       x
#       power
#           2
#           x
#   1

def powerf(x,y):
    return pow(x,y)

FUNCTIONS = [add, mul, powerf]
TERMINALS = ['x', 0, 1, 2]

main()

GENERATIONS = 100


dataset generated


In [ ]:
# FOR RESULTS SECTION OF MY REPORT....

GENERATIONS = 250
PROB_MUTATION = 0.4
XO_RATE = 0.8 
MAX_TREE_SIZE = 25
POP_SIZE = 120

# a valid tree for 'x * (2**x) + 1' is: (size=7)
# add
#   mul
#       x
#       power
#           2
#           x
#   1

def powerf(x,y):
    return pow(x,y)

FUNCTIONS = [add, mul, powerf]
TERMINALS = ['x', 0, 1, 2]

sol_found_in_gen = []
for i in range(20):
    sol_found_in_gen.append(main())

GENERATIONS = 100
print(sol_found_in_gen)

dataset generated
gen: 0
________________________
gen: 0 , best_of_run_f: 0.002 , best_of_run:
mul
   mul
      add
         x
         x
      add
         0
         x
   add
      mul
         x
         1
      x
gen: 1
________________________
gen: 1 , best_of_run_f: 0.143 , best_of_run:
add
   add
      mul
         powerf
            2
            x
         add
            0
            x
      add
         1
         2
   powerf
      2
      2
gen: 2
gen: 3
________________________
gen: 3 , best_of_run_f: 0.5 , best_of_run:
mul
   mul
      1
      x
   powerf
      2
      x
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
________________________
gen: 24 , best_of_run_f: 1.0 , best_of_run:
add
   mul
      powerf
         x
         1
      powerf
         2
         x
   powerf
      1
      1


_________________________________________________
END OF RUN
best_o

In [ ]:
print(sum(sol_found_in_gen) / len(sol_found_in_gen))

74.05


In [ ]:
# DEFAULT PARAMS:
# POP_SIZE        = 60   # population size
# MIN_DEPTH       = 2    # minimal initial random tree depth
# MAX_DEPTH       = 5    # maximal initial random tree depth
# GENERATIONS     = 100  # maximal number of generations to run evolution
# TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
# XO_RATE         = 0.8  # crossover rate 
# PROB_MUTATION   = 0.2  # per-node mutation probability 

# TUNING GP FOR MY NEEEDS

TERMINALS = ['x', -2, -1, 0, 1, 2]
FUNCTIONS = [add, mul, sub]
GENERATIONS = 150

# Testing if changing the terminals effects the outcome
TERMINALS = ['x', 0, 1, 2]
print("~~using terminals x, 0 ,1 ,2:")
main()
# result, produces a tree with the exact same fitness score!

# using 0.1 as a terminal:
TERMINALS = ['x', 0, 1, 0.1]
print("~~using terminals x, 0 , 1, 0.1:")
main()
# result?

# lower max depth than 5 throws errors!
# would like to add a permenant max depth, maybe set fitness to 0 if depth is above 10?
MAX_DEPTH = 5
# using power operator so the gp is capable of producing the actual sequence
# note: may need to write a condition to prevent overflow errors because of the power
def powerf(x,y):
    return pow(x,y)

FUNCTIONS = [add, mul, powerf]
print("~~using power function and terminals, 0, 1, 0.1:")
main()
# result

# providing ONLY the necessary terminals and functions
TERMINALS = ['x', 1, 2]
print("~~using power function and terminals x, 1, 2:")
main()

# setting terminals back to defaults
# TERMINALS = ['x', -2, -1, 0, 1, 2]
# FUNCTIONS = [add, mul, sub]
# changed my mind, want to run the rest of my experiments using the power function
GENERATIONS = 100

~~using terminals x, 0 ,1 ,2:
dataset generated
gen: 0
________________________
gen: 0 , best_of_run_f: 0.001 , best_of_run:
add
   sub
      mul
         x
         mul
            x
            x
      x
   add
      2
      x
gen: 1
________________________
gen: 1 , best_of_run_f: 0.001 , best_of_run:
sub
   mul
      x
      sub
         add
            0
            mul
               x
               x
         add
            sub
               0
               2
            sub
               0
               1
   add
      add
         0
         2
      1
gen: 2
________________________
gen: 2 , best_of_run_f: 0.002 , best_of_run:
mul
   sub
      sub
         add
            add
               sub
                  mul
                     1
                     0
                  sub
                     2
                     1
               2
            mul
               sub
                  x
                  2
               add
                  2
               

In [ ]:
# experiment changing mutation rate:
# default: PROB_MUTATION = 0.2
# note: need to change main to return run best fitness, for this code to work!
bests_fitnesses = []

for m_r in np.arange(0, 1.1, 0.1):
    PROB_MUTATION = m_r
    best_of_run = main()
    bests_fitnesses.append(best_of_run.fitness)
    print("obtained at mutation rate =", PROB_MUTATION)
    
print(bests_fitnesses)

# setting back to default
PROB_MUTATION = 0.2
    

dataset generated
gen: 0
________________________
gen: 0 , best_of_run_f: 0.001 , best_of_run:
mul
   mul
      add
         powerf
            x
            2
         add
            2
            2
      powerf
         powerf
            1
            2
         powerf
            x
            2
   add
      add
         1
         1
      mul
         2
         x
gen: 1
________________________
gen: 1 , best_of_run_f: 0.002 , best_of_run:
mul
   mul
      2
      powerf
         1
         x
   mul
      add
         mul
            mul
               x
               mul
                  x
                  1
            2
         2
      x
gen: 2
gen: 3
________________________
gen: 3 , best_of_run_f: 0.003 , best_of_run:
mul
   add
      mul
         x
         x
      add
         1
         add
            powerf
               2
               x
            x
   mul
      add
         2
         2
      2
gen: 4
________________________
gen: 4 , best_of_run_f: 0.196 , be

AttributeError: 'int' object has no attribute 'fitness'

In [ ]:
# experimenting with crossover rate

# default: XO_RATE = 0.8
bests_fitnesses = []
xo_rates = np.arange(0, 1.1, 0.1)

for xo_r in xo_rates:
    XO_RATE = xo_r
    best_of_run = main()
    bests_fitnesses.append(best_of_run.fitness)
    print("obtained at crossover rate =", XO_RATE)
    
print(bests_fitnesses)

# setting back to default
XO_RATE = 0.8

In [ ]:
# experimenting changing both crossover rate and mutation rates !
# todo: consider running this experiment 3 times and taking means...

# default: XO_RATE = 0.8
# default: PROB_MUTATION = 0.2
bests_fitnesses = []
xo_rates = np.arange(0, 1, 0.1)
mut_rates = np.arange(0, 1, 0.1)

for xo_r in xo_rates:
    current_xo_bests = []
    XO_RATE = xo_r
    for m_r in mut_rates:
        PROB_MUTATION = m_r
        best_of_run = main()
        current_xo_bests.append(best_of_run.fitness)
        print("obtained at crossover rate = {} and mutation rate = {}".format(XO_RATE, PROB_MUTATION))
        # todo: plot tree sizes and generations best was found.
        print("size of best tree =", best_of_run.size)
        print("found in generation = ", best_of_run.generation_found)
    bests_fitnesses.append(current_xo_bests)

# print fitnesses from each test
list(map(print, bests_fitnesses))


# setting back to default
XO_RATE = 0.8
PROB_MUTATION = 0.2

# Should I plot this?